# Calculate the Error Statistics of the Models Trained Using Evolving Time Windows

This notebook takes the composite projection files generated by the "train_and_run_mlp_models.ipynb" notebook and calculates the model error statistics for each model in one year blocks of time. The following statistical values, all computed using the sklearn package, are used to evaluate the MLP models:

| Parameter | Description | Documentation |
| :-: | :- | :-: |
| R2 | Coefficient of determination | [sklearn.metrics.r2_score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.r2_score.html) |
| RMS_ABS | Root-mean-squared of the absolute error | [sklearn.metrics.mean_squared_error](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html#sklearn.metrics.mean_squared_error) |
| RMS_NORM| The RMS_ABS value divided by the mean | [sklearn.metrics.mean_squared_error](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html#sklearn.metrics.mean_squared_error) |
| MAPE| Mean absolute percentage error | [sklearn.metrics.mean_absolute_percentage_error](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_absolute_percentage_error.html#sklearn.metrics.mean_absolute_percentage_error) |

In [4]:
# Start by importing the TELL package and information about your operating system:
import os 
import tell

import pandas as pd
import numpy as np

import sklearn
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_percentage_error


## Set the Directory Structure


In [5]:
# Identify the data input and output directories:
composite_input_directory = '/Users/burl878/Documents/Code/code_repos/burleyson-etal_2025_ldrd/data/composite_projections/'
statistics_output_directory = '/Users/burl878/Documents/Code/code_repos/burleyson-etal_2025_ldrd/data/'


## Calculate the Error Statistics by Year for Each BA


In [6]:
# Initiate a counter and empty dataframe to store the results:
counter = 0;
output_df = pd.DataFrame()

# Loop over the eight BAs used in this LDRD analysis:
for ba in ['AZPS', 'BPAT', 'CISO', 'ERCO', 'FPL', 'ISNE', 'PJM', 'WACM']:

    # Load in the compiled projection file:
    ba_df = pd.read_csv((composite_input_directory + ba + '_Composite_Data.csv'), index_col=None, header=0)

    # Replace the missing values with NaN:
    ba_df = ba_df.replace(-999.0, np.nan)

    # Convert the time to a datetime variable:
    ba_df['Time_UTC'] = pd.to_datetime(ba_df['Time_UTC'])

    # Extract the year from the Time_UTC variable:
    ba_df['Year'] = ba_df['Time_UTC'].dt.year  
    
    # Loop over all the model training windows:
    for model in ['M1', 'M2', 'M3', 'M4', 'M5', 'M6']:
    
        # Set the first forward year for each model:
        if model == 'M1':
           first_forward_year = 2018
        if model == 'M2':
           first_forward_year = 2019
        if model == 'M3':
           first_forward_year = 2020
        if model == 'M4':
           first_forward_year = 2021
        if model == 'M5':
           first_forward_year = 2022
        if model == 'M6':
           first_forward_year = 2023

        # Subset the data to just the variables needed for that model:
        subset_df = ba_df[['BA', 'Time_UTC', 'Demand_MWh', (model + '_MWh'), 'Year']].copy()

        # Loop over the years from the first forward year for that model through 2023:
        for year in range(first_forward_year,2024,1):

            # Iterate the counter by one:
            counter = counter + 1

            # Subset the data to an individual year:
            year_subset_df = subset_df.loc[(subset_df['Year'] == year)]

            # Drop all rows with missing values:
            year_subset_df = year_subset_df.dropna()

            # Calculate the difference between the prediction and observation:
            year_subset_df['Bias_MWh'] = year_subset_df[(model + '_MWh')] - year_subset_df['Demand_MWh']
            year_subset_df['Bias_%'] = 100*((year_subset_df[(model + '_MWh')] - year_subset_df['Demand_MWh']) / year_subset_df['Demand_MWh'])
            
            # Put the accuracy statistics into the output dataframe:
            output_df.loc[counter, 'BA'] = ba
            output_df.loc[counter, 'Model'] = model
            output_df.loc[counter, 'Year'] = int(year)
            output_df.loc[counter, 'Projection_Year'] = (year-first_forward_year)+1
            output_df.loc[counter, 'RMS_ABS'] = np.sqrt(mean_squared_error(year_subset_df[(model + '_MWh')], year_subset_df['Demand_MWh'])).round(3)
            output_df.loc[counter, 'RMS_NORM'] = ((np.sqrt(mean_squared_error(year_subset_df[(model + '_MWh')], year_subset_df['Demand_MWh'])).round(3)) / np.mean(year_subset_df['Demand_MWh'])).round(3)
            output_df.loc[counter, 'MAPE'] = mean_absolute_percentage_error(year_subset_df[(model + '_MWh')], year_subset_df['Demand_MWh']).round(3)
            output_df.loc[counter, 'R2'] = r2_score(year_subset_df[(model + '_MWh')], year_subset_df['Demand_MWh']).round(3)
            output_df.loc[counter, 'Max_Error_MWh'] = abs(year_subset_df['Bias_MWh']).max().round(1)
            output_df.loc[counter, 'Max_Error_%'] = abs(year_subset_df['Bias_%']).max().round(1)

            # Clean up and move to the next step in the loop:
            del year_subset_df

        # Clean up and move to the next step in the loop:
        del subset_df, first_forward_year

    # Clean up and move to the next step in the loop:
    del ba_df

# Convert the year variables to integers:
output_df['Year'] = output_df['Year'].round(0).astype(int)
output_df['Projection_Year'] = output_df['Projection_Year'].round(0).astype(int)

# Generate the .csv output file name:
csv_output_filename = (statistics_output_directory + 'Error_Statistics_Data.csv')
    
# Write out the dataframe to a .csv file:
output_df.to_csv(csv_output_filename, sep=',', index=False)

# Return the output_df:
output_df


NameError: name 'rms_abs' is not defined